## 設定POST 的Form Data

In [15]:
import requests

payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'103/04/29',
 'awardAnnounceEndDate':'103/04/29', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

## 送出Post 取得資料

In [37]:
user_post = requests.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8')  #為何要用encode ?? 用了encode ..好像沒差別
#print response_text

## 如何延續Session

In [36]:
#為何需要延續Session ?? 什麼情況之下 ??

rs = requests.session() 
rs_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
rs_get = rs.get("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2") 
response = rs_get.text.encode('utf8')
#print response


## 取得資料筆數

In [ ]:
from bs4 import BeautifulSoup

user_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload)
response_text = user_post.text.encode('utf8') 
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select('.T11b')[0]
#print int(rec_number_element.text) 
rec_number = int(rec_number_element.text) 
#print rec_number

## 將資料筆數轉換成頁數

In [35]:
#-----------------------------------------------------
from math import ceil 
#int(ceil(float(rec_number) / 100))
page_number = int(ceil(float(rec_number) / 100)) 
#print page_number


## 字串格式化 - 使用format

In [40]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex={0}" 
for page in range(1, 10 + 1): 
    print page_format.format(page)

#--------------------------------------------
# 使用%
#page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
#for page in range(1, 5 + 1): 
#    print page_format%(page)
#--------------------------------------------

IndentationError: expected an indented block (<ipython-input-40-a1bfaf079a51>, line 10)

## 取得每頁標案清單的內容

In [43]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8')
    #print bid_response

## 先拿一頁做試驗

In [45]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
page = 1 
bid_list = rs.get(page_format%(page)) 
bid_response = bid_list.text.encode('utf8') 
bid_soup = BeautifulSoup(bid_response) 
#print bid_response
bid_table = bid_soup.select('#print_area')[0]
#print bid_table

## 去掉標頭 去掉標頭跟頁次 索引範例

In [47]:
bid_rows = bid_table.select('tr')[1:] 
#print bid_rows
bid_rows = bid_table.select('tr')[1:-1] 
#print bid_rows
ary = [1,2,3,4,5,6] 
print ary[1:-1] 
#[2,3,4,5]

[2, 3, 4, 5]


## 抓出每一列所有的連結  (重點!!!!)

In [56]:
#print bid_table
bid_rows = bid_table.select('tr')[1:-1] 
for bid_row in bid_rows: 
    links = [tag['href']for tag in bid_row.select('a')][0]  # 為何是用tag ??????????? ...取第一個連結
    #print links

## 取得實際連結

In [73]:
for bid_row in bid_rows: 
    link = [tag['href'] for tag in bid_row.findAll('a',{'href': True})][0]   # 為何是用True?????
    link_href = "http://web.pcc.gov.tw/tps" + link[2:]  # [2:] 去除前面兩個字元
    print link_href

http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239371&tenderCaseNo=4197
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51241675&tenderCaseNo=XW03118P099
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51241674&tenderCaseNo=XW03072P119
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51241666&tenderCaseNo=B1030411
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51241665&tenderCaseNo=B1030422
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atm